## CAD-Recode Demonstration
In this demonstration, we load a point cloud and process it using a pre-trained CAD-Recode model, which generates Python CadQuery code. Finally, we interpret the code to create a CAD model, visualize it, and evaluate its quality metrics. This notebook works on both CPU and GPU.

### Define CAD-Recode model

In [12]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "2"

In [ ]:
import open3d
import trimesh
import skimage.io
import numpy as np
import cadquery as cq
import matplotlib.pyplot as plt
from scipy.spatial import cKDTree

import torch
from torch import nn
from transformers import (
    AutoTokenizer, Qwen3ForCausalLM, Qwen3Model, PreTrainedModel, AutoModelForCausalLM)
from transformers.modeling_outputs import CausalLMOutputWithPast
from pytorch3d.ops import sample_farthest_points
from peft import PeftModel, LoraConfig, get_peft_model


class FourierPointEncoder(nn.Module):
    def __init__(self, hidden_size):
        super().__init__()
        frequencies = 2.0 ** torch.arange(8, dtype=torch.float32)
        self.register_buffer('frequencies', frequencies, persistent=False)
        self.projection = nn.Linear(51, hidden_size)

    def forward(self, points):
        x = points
        x = (x.unsqueeze(-1) * self.frequencies).view(*x.shape[:-1], -1)
        x = torch.cat((points, x.sin(), x.cos()), dim=-1)
        x = x.to(self.projection.weight.dtype) 
        x = self.projection(x)
        return x
        # points shape: (batch_size, num_points, 3)



class CADRecode(Qwen3ForCausalLM):
    def __init__(self, config):
        PreTrainedModel.__init__(self, config)
        self.model = Qwen3Model(config)
        self.vocab_size = config.vocab_size
        self.lm_head = nn.Linear(config.hidden_size, config.vocab_size, bias=False)
        
        torch.set_default_dtype(torch.float32)
        self.point_encoder = FourierPointEncoder(config.hidden_size)
        torch.set_default_dtype(torch.bfloat16)

    def forward(self,
                input_ids=None,
                attention_mask=None,
                point_cloud=None,
                position_ids=None,
                past_key_values=None,
                inputs_embeds=None,
                labels=None,
                use_cache=None,
                output_attentions=None,
                output_hidden_states=None,
                return_dict=None,
                cache_position=None):
        output_attentions = output_attentions if output_attentions is not None else self.config.output_attentions
        output_hidden_states = output_hidden_states if output_hidden_states is not None else self.config.output_hidden_states
        return_dict = return_dict if return_dict is not None else self.config.use_return_dict

        # concatenate point and text embeddings
        if past_key_values is None or past_key_values.get_seq_length() == 0:
            assert inputs_embeds is None
            inputs_embeds = self.model.embed_tokens(input_ids)
            point_embeds = self.point_encoder(point_cloud).bfloat16()
            inputs_embeds[attention_mask == -1] = point_embeds.reshape(-1, point_embeds.shape[2])
            attention_mask[attention_mask == -1] = 1
            input_ids = None
            position_ids = None

        # decoder outputs consists of (dec_features, layer_state, dec_hidden, dec_attn)
        outputs = self.model(
            input_ids=input_ids,
            attention_mask=attention_mask,
            position_ids=position_ids,
            past_key_values=past_key_values,
            inputs_embeds=inputs_embeds,
            use_cache=use_cache,
            output_attentions=output_attentions,
            output_hidden_states=output_hidden_states,
            return_dict=return_dict,
            cache_position=cache_position)

        hidden_states = outputs[0]
        logits = self.lm_head(hidden_states)
        logits = logits.float()

        loss = None
        if labels is not None:
            # Shift so that tokens < n predict n
            shift_logits = logits[..., :-1, :].contiguous()
            shift_labels = labels[..., 1:].contiguous()
            # Flatten the tokens
            loss_fct = nn.CrossEntropyLoss()
            shift_logits = shift_logits.view(-1, self.config.vocab_size)
            shift_labels = shift_labels.view(-1)
            # Enable model parallelism
            shift_labels = shift_labels.to(shift_logits.device)
            loss = loss_fct(shift_logits, shift_labels)

        if not return_dict:
            output = (logits,) + outputs[1:]
            return (loss,) + output if loss is not None else output

        return CausalLMOutputWithPast(
            loss=loss,
            logits=logits,
            past_key_values=outputs.past_key_values,
            hidden_states=outputs.hidden_states,
            attentions=outputs.attentions)

    def prepare_inputs_for_generation(self, *args, **kwargs):
        model_inputs = super().prepare_inputs_for_generation(*args, **kwargs)
        model_inputs['point_cloud'] = kwargs['point_cloud']
        return model_inputs

### Load CAD-Recode checkpoint

In [ ]:
BASE_MODEL_PATH = '/home/user2/wns/cad-recode/Qwen3-1.7B'
CHECKPOINT_DIR = '/home/user2/wns/cad-recode/checkpoint-65400' 
POINT_ENCODER_WEIGHTS_PATH = os.path.join(CHECKPOINT_DIR, "point_encoder_weights.bin")

device = 'cuda' if torch.cuda.is_available() else 'cpu'
attn_implementation = 'flash_attention_2' if torch.cuda.is_available() else None

print(f"Loading full model from checkpoint: {CHECKPOINT_DIR}")

tokenizer = AutoTokenizer.from_pretrained(
    BASE_MODEL_PATH,
    pad_token='<|im_end|>',
    padding_side='left',
    trust_remote_code=True
)
print(f"Tokenizer loaded from: {BASE_MODEL_PATH}")

# 1. 加载原始的基础 LLM 模型 (Qwen3ForCausalLM)
print(f"Loading base LLM (Qwen3ForCausalLM) from: {BASE_MODEL_PATH}")
base_llm = AutoModelForCausalLM.from_pretrained(
    BASE_MODEL_PATH,
    torch_dtype=torch.bfloat16, 
    attn_implementation=attn_implementation,
    trust_remote_code=True
)
print("Base LLM loaded.")

# 2. 实例化你的 CADRecode 模型。
# 此时，model 的 Qwen 部分和 point_encoder 都是随机初始化的。
print("Instantiating CADRecode model structure (initially random weights for Qwen and point_encoder)...")
model = CADRecode(base_llm.config) 

# 3. 将加载了预训练权重的 base_llm 的核心 Qwen 结构赋值给 CADRecode 实例的相应属性。
# 这样 CADRecode 的 Qwen 部分就有了预训练权重。
model.model = base_llm.model # 赋值 Qwen3Model 实例
model.lm_head = base_llm.lm_head # 赋值 Qwen3 的语言模型头
print("Base LLM's core model and lm_head assigned to CADRecode instance.")

# 4. 定义 PEFT LoRA 配置。
# **关键修改：明确指定 target_modules，排除 point_encoder 的线性层。**
# Qwen 系列模型的线性层名称通常是：q_proj, k_proj, v_proj, o_proj (注意力层),
# 以及 gate_proj, up_proj, down_proj (MLP层)。
# 请确保这个列表涵盖了 Qwen3 中所有需要 LoRA 的线性层，但不包含你的 `point_encoder` 中的线性层。
peft_config_for_loading = LoraConfig(
    r=64,
    lora_alpha=16,
    target_modules=[
        "q_proj", "k_proj", "v_proj", "o_proj", 
        "gate_proj", "up_proj", "down_proj"
    ], 
    lora_dropout=0.1,
    bias="none",
    task_type="CAUSAL_LM",
)

# 5. **将整个 CADRecode 实例应用 PEFT LoRA。**
# 这一步会返回一个 `PeftModelForCausalLM` 实例，它包装了 `CADRecode`。
# 由于 `target_modules` 已排除 point_encoder，所以 `point_encoder` 将不会被 LoRA 化。
print("Applying PEFT LoRA to the CADRecode model structure (only Qwen layers will be wrapped)...")
model = get_peft_model(model, peft_config_for_loading) 
print("CADRecode model (now a PeftModelForCausalLM) prepared.")

# 6. **加载 LoRA 权重。**
# `model` 现在是 `PeftModelForCausalLM` 的实例。
# `PeftModel` 的 `load_adapter` 方法会从指定目录加载 `adapter_model.safetensors` 并应用到 `model` 中。
print(f"Loading LoRA adapters for base LLM layers from: {CHECKPOINT_DIR}")
model.load_adapter(CHECKPOINT_DIR, adapter_name="default")
print("LoRA adapters for base LLM layers loaded.")

# 7. **手动加载 point_encoder 的权重。**
# `model` 是 `PeftModelForCausalLM`，访问其原始 `CADRecode` 实例需要 `model.base_model.model`。
# 由于 `point_encoder` 现在没有被 LoRA 化，`model.base_model.model.point_encoder` 是一个标准的 `FourierPointEncoder`。
# 它期望 `point_encoder_state_dict` 包含 `projection.weight` 和 `projection.bias`。
print(f"Attempting to load point_encoder weights from: {POINT_ENCODER_WEIGHTS_PATH}")
if os.path.exists(POINT_ENCODER_WEIGHTS_PATH):
    point_encoder_state_dict = torch.load(POINT_ENCODER_WEIGHTS_PATH, map_location='cpu')
    
    # 这一行应该能成功，因为现在 point_encoder 是非 LoRA 化的，
    # 并且你确认保存的权重也是非 LoRA 化的。
    model.base_model.model.point_encoder.load_state_dict(point_encoder_state_dict, strict=True) 
    print("Point_encoder weights loaded successfully.")
else:
    print(f"Error: {POINT_ENCODER_WEIGHTS_PATH} not found. 'point_encoder' will be randomly initialized.")
    print("Please ensure the specified CHECKPOINT_DIR contains 'point_encoder_weights.bin'.")

# 8. 将整个 CADRecode 模型（现在包含所有组件）移动到设备并设置为评估模式
# .to(device) 应该在所有权重加载完成后进行，以确保所有模块都在正确的设备上。
model.eval().to(device)
print("Full model (CADRecode with Qwen3-1.7B + LoRA + trained point_encoder) ready for inference.")

### Load input point cloud
Load the mesh of one of the examples from the DeepCAD test set and sample 256 points from it. Then input mesh is normalized to fit within a cube of size 2, centered at the origin. Both the mesh and point cloud can be visualized interactively in this notebook using `open3d` or `trimesh`; however, for non-interactive environments, we render them as static images.

In [ ]:
!wget https://github.com/filaPro/cad-recode/releases/download/v1.0/00409751.stl

In [15]:
def mesh_to_point_cloud(mesh, n_points=256, n_pre_points=8192):
    vertices, _ = trimesh.sample.sample_surface(mesh, n_pre_points)
    _, ids = sample_farthest_points(torch.tensor(vertices).unsqueeze(0), K=n_points)
    ids = ids[0].numpy()
    return np.asarray(vertices[ids])


gt_mesh = trimesh.load_mesh('./00409751.stl')
gt_mesh.apply_translation(-(gt_mesh.bounds[0] + gt_mesh.bounds[1]) / 2.0)
gt_mesh.apply_scale(2.0 / max(gt_mesh.extents))
np.random.seed(0)
point_cloud = mesh_to_point_cloud(gt_mesh)

### Let's have a look at input point cloud
It looks like a table with four legs!

In [16]:
def render(geometry, image_size=128, camera_distance=-1.8, image_path='/home/user2/wns/cad-recode/tmp/tmp.png'):
    visualizer = open3d.visualization.Visualizer()
    visualizer.create_window(visible=False)
    visualizer.add_geometry(geometry)

    lookat = np.array([0, 0, 0], dtype=np.float32)
    front = np.array([1, 1, 1], dtype=np.float32)
    up = np.array([0, 1, 0], dtype=np.float32)
    eye = lookat + front * camera_distance
    right = np.cross(up, front)
    right /= np.linalg.norm(right)
    true_up = np.cross(front, right)
    rotation_matrix = np.column_stack((right, true_up, front)).T
    extrinsic = np.eye(4)
    extrinsic[:3, :3] = rotation_matrix
    extrinsic[:3, 3] = -rotation_matrix @ eye

    view_control = visualizer.get_view_control()
    camera_params = view_control.convert_to_pinhole_camera_parameters()
    camera_params.extrinsic = extrinsic
    view_control.convert_from_pinhole_camera_parameters(camera_params)

    visualizer.poll_events()
    visualizer.update_renderer()
    visualizer.capture_screen_image(image_path)
    visualizer.destroy_window()

    image = skimage.io.imread(image_path)
    image = skimage.transform.resize(
        image,
        output_shape=(image_size, image_size),
        order=2,
        anti_aliasing=True,
        preserve_range=True).astype(np.uint8)
    return image

pcd = open3d.geometry.PointCloud()
pcd.points = open3d.utility.Vector3dVector(point_cloud)
pcd.paint_uniform_color(np.array([0, 80, 239]) / 255)
plt.figure(figsize=(3, 3))
_ = plt.imshow(render(pcd))


[Open3D WARNING] Failed to initialize GLEW.


AttributeError: 'NoneType' object has no attribute 'convert_to_pinhole_camera_parameters'

<Figure size 300x300 with 0 Axes>

### Run CAD-Recode on the input point cloud

In [17]:
input_ids = [tokenizer.pad_token_id] * len(point_cloud) + [tokenizer('<|im_start|>')['input_ids'][0]]
attention_mask = [-1] * len(point_cloud) + [1]
with torch.no_grad():
    batch_ids = model.generate(
        input_ids=torch.tensor(input_ids).unsqueeze(0).to(model.device),
        attention_mask=torch.tensor(attention_mask).unsqueeze(0).to(model.device),
        point_cloud=torch.tensor(point_cloud.astype(np.float32)).unsqueeze(0).to(model.device),
        max_new_tokens=768,
        pad_token_id=tokenizer.pad_token_id)
py_string = tokenizer.batch_decode(batch_ids)[0]
begin = py_string.find('<|im_start|>') + 12
end = py_string.find('<|endoftext|>')
py_string = py_string[begin: end]
print(py_string)

a khúc D S S T RZ................................................... ............... ... ... ............ ......... ... ....................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................... ... ... ... ... ... ... ... ... ... ... ... ... ... ... ... ... ... ... ... ... ... ... ... ... ... ... ... ... ... ... 

### Execute predicted python code to raise a CAD model
The predicted code might be invalid or could potentially cause memory leaks in CadQuery, as described in this [issue](https://github.com/CadQuery/cadquery/issues/1665).  Therefore, it is recommended to run the next cell in a separate process with a timeout of, for example, 3 seconds.:
```
process = Process(target=..., args=...)
process.start()
process.join(3)

if process.is_alive():
    process.terminate()
    process.join()
```
But in this demo safety is omitted.

In [11]:
exec(py_string, globals())
compound = globals()['r'].val()
vertices, faces = compound.tessellate(0.001, 0.1)
mesh = trimesh.Trimesh([(v.x, v.y, v.z) for v in vertices], faces)
mesh.export('/home/user2/wns/cad-recode/tmp/333.stl')
cq.exporters.export(compound, '/home/user2/wns/cad-recode/tmp/333.step')

SyntaxError: unmatched ')' (<string>, line 4)

In [9]:
mesh = open3d.io.read_triangle_mesh('/home/user2/wns/cad-recode/tmp/1.stl')
mesh.vertices = open3d.utility.Vector3dVector(np.asarray(mesh.vertices) / 100.)
mesh.paint_uniform_color(np.array([255, 255, 136]) / 255)
mesh.compute_vertex_normals()
plt.figure(figsize=(3, 3))
_ = plt.imshow(render(mesh))

[Open3D WARNING] Failed to initialize GLEW.


AttributeError: 'NoneType' object has no attribute 'convert_to_pinhole_camera_parameters'

<Figure size 300x300 with 0 Axes>

### Compute IoU and Chamfer distance metrics
The predicted model resembles a table, but let's calculate how closely it matches the ground truth mesh.

In [10]:
pred_mesh = trimesh.load_mesh('/home/user2/wns/cad-recode/tmp/1.stl')
pred_mesh.apply_transform(trimesh.transformations.scale_matrix(1 / 100 / 2))
pred_mesh.apply_transform(trimesh.transformations.translation_matrix([0.5, 0.5, 0.5]))
gt_mesh.apply_transform(trimesh.transformations.scale_matrix(1 / 2))
gt_mesh.apply_transform(trimesh.transformations.translation_matrix([0.5, 0.5, 0.5]))

n_points = 8192
gt_points, _ = trimesh.sample.sample_surface(gt_mesh, n_points)
pred_points, _ = trimesh.sample.sample_surface(pred_mesh, n_points)
gt_distance, _ = cKDTree(gt_points).query(pred_points, k=1)
pred_distance, _ = cKDTree(pred_points).query(gt_points, k=1)
cd = np.mean(np.square(gt_distance)) + np.mean(np.square(pred_distance))

intersection_volume = 0
for gt_mesh_i in gt_mesh.split():
    for pred_mesh_i in pred_mesh.split():
        intersection = gt_mesh_i.intersection(pred_mesh_i)
        volume = intersection.volume if intersection is not None else 0
        intersection_volume += volume

gt_volume = sum(m.volume for m in gt_mesh.split())
pred_volume = sum(m.volume for m in pred_mesh.split())
union_volume = gt_volume + pred_volume - intersection_volume
iou = intersection_volume / union_volume

print(f'CD: {cd * 1000:.3f}, IoU: {iou:.3f}')

CD: 0.283, IoU: 0.943
